In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/October_missions_full.csv')
df['mission'] = df['type'] + '_' + df['target'].astype(str)

df = df[['user', 'mission', 'createdAtT', 'type', 'target', 'completed', 'performance']]
df['createdAtT'] = pd.to_datetime(df['createdAtT'], unit='ms')
df['mission'] = df['mission'].astype('category')
df['type'] = df['type'].astype('category')

mean_size = df.groupby('user').size().mean()
df = df.groupby('user').filter(lambda x: len(x) >= mean_size)
df['user'] = df['user'].astype('category').cat.codes
df.sort_values(by=['createdAtT'], inplace=True, ignore_index=True)
df

,user,mission,createdAtT,type,target,completed,performance
0,471,mobility_1,2024-10-01 00:00:17.857,mobility,1,False,0.000000
1,471,quiz_6,2024-10-01 00:00:17.857,quiz,6,True,1.000000
2,471,episode_3,2024-10-01 00:00:17.857,episode,3,False,0.666667
3,888,quiz_1,2024-10-01 00:00:21.680,quiz,1,True,1.000000
4,888,episode_2,2024-10-01 00:00:21.680,episode,2,True,1.000000
...,...,...,...,...,...,...,...
66063,862,activity_2,2024-10-31 23:51:28.023,activity,2,True,1.000000
66064,862,quiz_2,2024-10-31 23:51:28.023,quiz,2,True,1.000000
66065,923,mobility_2,2024-10-31 23:55:25.309,mobility,2,False,0.000000
66066,923,episode_1,2024-10-31 23:55:25.309,episode,1,False,0.000000
